In [1]:
!pip install tensorflow 

In [2]:
import tensorflow
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)


In [4]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [5]:
valid_datagen=ImageDataGenerator(rescale=1./255)

In [6]:
import zipfile

zip_path = "Chest X-ray Dataset Kaggle.zip"
extract_path = "Chest X-ray Dataset Kaggle"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extraction done!")


✅ Extraction done!


In [7]:
train_generator = train_datagen.flow_from_directory(
    "Chest X-ray Dataset Kaggle/chest_xray/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
)


Found 5216 images belonging to 2 classes.


In [8]:
test_generator=test_datagen.flow_from_directory(
    "Chest X-ray Dataset Kaggle/chest_xray/test",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
)

Found 625 images belonging to 2 classes.


In [9]:
valid_generator=valid_datagen.flow_from_directory(
    "Chest X-ray Dataset kaggle/chest_xray/val",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
)

Found 16 images belonging to 2 classes.


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense

class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(MultiHeadSelfAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)
        self.projection_dim = self.embed_dim // self.num_heads

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        attention, _ = self.attention(query, key, value)
        attention = tf.reshape(attention, (batch_size, seq_len, self.embed_dim))
        output = self.combine_heads(attention)
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [11]:
from keras.layers import LayerNormalization, Dropout, Layer, Dense
from keras.models import Sequential

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ffn_dim, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ffn_dim = ffn_dim

        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = Sequential([
            Dense(ffn_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate=0.01)
        self.dropout2 = Dropout(rate=0.01)

    def call(self, inputs, training):
        attention = self.att(inputs)
        attention = self.dropout1(attention, training=training)
        out1 = self.layernorm1(inputs + attention)
        out2 = self.ffn(out1)
        out2 = self.dropout2(out2, training=training)
        return self.layernorm2(out1 + out2)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ffn_dim": self.ffn_dim
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [12]:
from keras.layers import Layer, Dense

class PatchEmbedding(Layer):
    def __init__(self, num_patches, embed_dim, **kwargs):
        super(PatchEmbedding, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.projection = Dense(embed_dim)

    def call(self, patches):
        return self.projection(patches)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "embed_dim": self.embed_dim
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [13]:
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalAveragePooling1D, Conv2D, MaxPooling2D
from keras.models import Model
import tensorflow as tf

class VisionImageTransformer(Model):
    def __init__(self, num_patches, num_heads, embed_dim, ffn_dim, num_layers, num_classes, **kwargs):
        super(VisionImageTransformer, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.ffn_dim = ffn_dim
        self.num_layers = num_layers
        self.num_classes = num_classes

        self.patch_embed = PatchEmbedding(num_patches, embed_dim)
        self.transformer_layers = [
            TransformerBlock(embed_dim, num_heads, ffn_dim)
            for _ in range(num_layers)
        ]

        self.flatten = Flatten()
        self.conv2D_layer1 = Conv2D(16, (3, 3), strides=(1, 1), activation='relu')
        self.mpool_layer1 = MaxPooling2D(pool_size=(2, 2))
        self.conv2D_layer2 = Conv2D(16, (3, 3), strides=(1, 1), activation='relu')
        self.mpool_layer2 = MaxPooling2D(pool_size=(2, 2))
        self.dense = Dense(1, activation='sigmoid')
        self.pool = GlobalAveragePooling1D()
        self.dropout = Dropout(rate=0.5)

    def call(self, inputs, training=False):
        inp_preprocessed = self.conv2D_layer1(inputs)
        inp_preprocessed = self.mpool_layer1(inp_preprocessed)
        inp_preprocessed = self.conv2D_layer2(inp_preprocessed)
        inp_preprocessed = self.mpool_layer2(inp_preprocessed)

        patches = tf.image.extract_patches(
            images=inp_preprocessed,
            sizes=[1, 16, 16, 1],
            strides=[1, 16, 16, 1],
            rates=[1, 1, 1, 1],
            padding='SAME'
        )
        batch_size = tf.shape(patches)[0]
        patches = tf.reshape(patches, [batch_size, -1, tf.shape(patches)[-1]])

        x = self.patch_embed(patches)
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x, training=training)

        x = self.dropout(x, training=training)
        x = self.pool(x)
        return self.dense(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "num_heads": self.num_heads,
            "embed_dim": self.embed_dim,
            "ffn_dim": self.ffn_dim,
            "num_layers": self.num_layers,
            "num_classes": self.num_classes
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [14]:
import tensorflow.keras
from tensorflow.keras.optimizers import Adam
model=VisionImageTransformer(
    num_patches=196,
    embed_dim=100,
    num_heads=4,
    ffn_dim=130,
    num_layers=4,
    num_classes=2
)
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
model.fit(train_generator,validation_data=test_generator,epochs=30,callbacks=[early_stop])

C:\Users\Rajveer Gupta\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


C:\Users\Rajveer Gupta\anaconda3\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'vision_image_transformer', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


163/163 ━━━━━━━━━━━━━━━━━━━━ 263s 1s/step - accuracy: 0.7225 - loss: 0.6169 - val_accuracy: 0.6240 - val_loss: 0.9060
Epoch 2/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 153s 819ms/step - accuracy: 0.7694 - loss: 0.4663 - val_accuracy: 0.6544 - val_loss: 0.6981
Epoch 3/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 156s 949ms/step - accuracy: 0.8211 - loss: 0.3715 - val_accuracy: 0.6608 - val_loss: 0.6763
Epoch 4/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 1114s 7s/step - accuracy: 0.8421 - loss: 0.3289 - val_accuracy: 0.8000 - val_loss: 0.4773
Epoch 5/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 11354s 70s/step - accuracy: 0.8533 - loss: 0.3170 - val_accuracy: 0.7568 - val_loss: 0.4606
Epoch 6/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.8613 - loss: 0.2925 - val_accuracy: 0.7920 - val_loss: 0.4704
Epoch 7/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.8805 - loss: 0.2814 - val_accuracy: 0.8224 - val_loss: 0.4055
Epoch 8/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 715s 4s/step - accuracy: 0.8832 - loss: 0.2710 - val_accur

In [19]:
model.save('Pneumonia_Detector_Model_2.keras')

In [20]:
accuracy=model.evaluate(valid_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.7500 - loss: 0.7044
